In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
import os
from itertools import product
import math
import numpy
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import scipy
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import dendrogram, linkage, leaves_list
from scipy.spatial.distance import pdist
from sklearn.manifold import TSNE, MDS
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time

sys.path.append(os.path.abspath(".."))
from distance_functions_final import *

def symmetrize(A):
    n = A.shape[0]
    B = A.copy()
    B[np.tril_indices(n)] = B.T[np.tril_indices(n)]
    return B

all_classes = {}

all_classes["resnet"] = ['regnet_x_16gf', 'regnet_x_1_6gf', 'regnet_x_32gf', 'regnet_x_3_2gf', 'regnet_x_400mf',
            'regnet_x_800mf', 'regnet_x_8gf', 'regnet_y_16gf', 'regnet_y_1_6gf', 'regnet_y_32gf',
            'regnet_y_3_2gf', 'regnet_y_400mf', 'regnet_y_800mf', 'regnet_y_8gf', 'resnet18',
            'resnext50_32x4d', 'wide_resnet50_2'] # convolutional residual networks (skip connections) # 17

all_classes["efficientnet"] = ['efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4',
            'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7'] # convolutional # 8

all_classes["alexnet"] = ["alexnet"] # convolutional # 1

all_classes["googlenet"] = ["googlenet"] # convolutional # 1

all_classes["inception"] = ["inception"] # convolutional # 1

all_classes["mnasnet"] = ["mnasnet"] # convolutional # 1

all_classes["vgg16"] = ["vgg16"] # convolutional # 1

all_classes["mobilenet"] = ['mobilenet_v2', 'mobilenet_v3_large', 'mobilenet_v3_small'] # convolutional # 3

all_classes["convnext"] = ['convnext_small', 'convnext_tiny'] # depthwise convolutional that were designed to compete with transformers # 2

label_names_dict = {'lin_cka_dist': 'lin CKA', 'mean_sq_cca_e2e' : 'CCA', 'pwcca_dist_e2e': 'PWCCA', 'procrustes': 'Procrustes'}
label_names_dict['GULP_dist_0.000000e+00'] = 'GULP, $\lambda = 0$'
label_names_dict['GULP_dist_1.000000e-07'] = 'GULP, $\lambda = 10^{-7}$'
label_names_dict['GULP_dist_1.000000e-06'] = 'GULP, $\lambda = 10^{-6}$'
label_names_dict['GULP_dist_1.000000e-05'] = 'GULP, $\lambda = 10^{-5}$'
label_names_dict['GULP_dist_1.000000e-04'] = 'GULP, $\lambda = 10^{-4}$'
label_names_dict['GULP_dist_1.000000e-03'] = 'GULP, $\lambda = 10^{-3}$'
label_names_dict['GULP_dist_1.000000e-02'] = 'GULP, $\lambda = 10^{-2}$'
label_names_dict['GULP_dist_1.000000e-01'] = 'GULP, $\lambda = 10^{-1}$'
label_names_dict['GULP_dist_1.000000e+00'] = 'GULP, $\lambda = 1$'
label_names_dict['GULP_dist_1.000000e+01'] = 'GULP, $\lambda = 10$' # GULP lambda = 10

label_names_dict['CKA_dist_RBF_1.000000e-03'] = 'CKA_RBF, $\sigma= 10^{-3}$'
label_names_dict['CKA_dist_RBF_1.000000e-02'] = 'CKA_RBF, $\sigma= 10^{-2}$'
label_names_dict['CKA_dist_RBF_1.000000e-01'] = 'CKA_RBF, $\sigma= 10^{-1}$'
label_names_dict['CKA_dist_RBF_1.000000e+00'] = 'CKA_RBF, $\sigma = 1$'
label_names_dict['CKA_dist_RBF_1.000000e+01'] = 'CKA_RBF, $\sigma = 10$'
label_names_dict['CKA_dist_RBF_1.000000e+02'] = 'CKA_RBF, $\sigma = 100$'

label_names_dict['CKA_dist_Laplace_0.001'] = 'CKA_Lap, $\sigma= 10^{-3}$'
label_names_dict['CKA_dist_Laplace_0.01'] = 'CKA_Lap, $\sigma= 10^{-2}$'
label_names_dict['CKA_dist_Laplace_0.09999999999999999'] = 'CKA_Lap, $\sigma= 10^{-1}$'
label_names_dict['CKA_dist_Laplace_1.0'] = 'CKA_Lap, $\sigma = 1$'
label_names_dict['CKA_dist_Laplace_10.0'] = 'CKA_Lap, $\sigma = 10$'
label_names_dict['CKA_dist_Laplace_100.0'] = 'CKA_Lap, $\sigma = 100$'

label_names_dict['UKP_dist_RBF_1.000000e-03_1.000000e-07'] = 'UKP_RBF, $\lambda = 10^{-3},\sigma= 10^{-7}$'
label_names_dict['UKP_dist_RBF_1.000000e-02_1.000000e-07'] = 'UKP_RBF, $\lambda = 10^{-2},\sigma= 10^{-7}$'
label_names_dict['UKP_dist_RBF_1.000000e-01_1.000000e-07'] = 'UKP_RBF, $\lambda = 10^{-1},\sigma= 10^{-7}$'
label_names_dict['UKP_dist_RBF_1.000000e+00_1.000000e-07'] = 'UKP_RBF, $\lambda = 1,\sigma= 10^{-7}$'
label_names_dict['UKP_dist_RBF_1.000000e+01_1.000000e-07'] = 'UKP_RBF, $\lambda = 10,\sigma= 10^{-7}$'
label_names_dict['UKP_dist_RBF_1.000000e+02_1.000000e-07'] = 'UKP_RBF, $\lambda = 100,\sigma= 10^{-7}$'

label_names_dict['UKP_dist_RBF_1.000000e-03_1.000000e-06'] = 'UKP_RBF, $\lambda = 10^{-3},\sigma= 10^{-6}$'
label_names_dict['UKP_dist_RBF_1.000000e-02_1.000000e-06'] = 'UKP_RBF, $\lambda = 10^{-2},\sigma= 10^{-6}$'
label_names_dict['UKP_dist_RBF_1.000000e-01_1.000000e-06'] = 'UKP_RBF, $\lambda = 10^{-1},\sigma= 10^{-6}$'
label_names_dict['UKP_dist_RBF_1.000000e+00_1.000000e-06'] = 'UKP_RBF, $\lambda = 1,\sigma= 10^{-6}$'
label_names_dict['UKP_dist_RBF_1.000000e+01_1.000000e-06'] = 'UKP_RBF, $\lambda = 10,\sigma= 10^{-6}$'
label_names_dict['UKP_dist_RBF_1.000000e+02_1.000000e-06'] = 'UKP_RBF, $\lambda = 100,\sigma= 10^{-6}$'

label_names_dict['UKP_dist_RBF_1.000000e-03_1.000000e-05'] = 'UKP_RBF, $\lambda = 10^{-3},\sigma= 10^{-5}$'
label_names_dict['UKP_dist_RBF_1.000000e-02_1.000000e-05'] = 'UKP_RBF, $\lambda = 10^{-2},\sigma= 10^{-5}$'
label_names_dict['UKP_dist_RBF_1.000000e-01_1.000000e-05'] = 'UKP_RBF, $\lambda = 10^{-1},\sigma= 10^{-5}$'
label_names_dict['UKP_dist_RBF_1.000000e+00_1.000000e-05'] = 'UKP_RBF, $\lambda = 1,\sigma= 10^{-5}$'
label_names_dict['UKP_dist_RBF_1.000000e+01_1.000000e-05'] = 'UKP_RBF, $\lambda = 10,\sigma= 10^{-5}$'
label_names_dict['UKP_dist_RBF_1.000000e+02_1.000000e-05'] = 'UKP_RBF, $\lambda = 100,\sigma= 10^{-5}$'

label_names_dict['UKP_dist_RBF_1.000000e-03_1.000000e-04'] = 'UKP_RBF, $\lambda = 10^{-3},\sigma= 10^{-4}$'
label_names_dict['UKP_dist_RBF_1.000000e-02_1.000000e-04'] = 'UKP_RBF, $\lambda = 10^{-2},\sigma= 10^{-4}$'
label_names_dict['UKP_dist_RBF_1.000000e-01_1.000000e-04'] = 'UKP_RBF, $\lambda = 10^{-1},\sigma= 10^{-4}$'
label_names_dict['UKP_dist_RBF_1.000000e+00_1.000000e-04'] = 'UKP_RBF, $\lambda = 1,\sigma= 10^{-4}$'
label_names_dict['UKP_dist_RBF_1.000000e+01_1.000000e-04'] = 'UKP_RBF, $\lambda = 10,\sigma= 10^{-4}$'
label_names_dict['UKP_dist_RBF_1.000000e+02_1.000000e-04'] = 'UKP_RBF, $\lambda = 100,\sigma= 10^{-4}$'

label_names_dict['UKP_dist_RBF_1.000000e-03_1.000000e-03'] = 'UKP_RBF, $\lambda = 10^{-3},\sigma= 10^{-3}$'
label_names_dict['UKP_dist_RBF_1.000000e-02_1.000000e-03'] = 'UKP_RBF, $\lambda = 10^{-2},\sigma= 10^{-3}$'
label_names_dict['UKP_dist_RBF_1.000000e-01_1.000000e-03'] = 'UKP_RBF, $\lambda = 10^{-1},\sigma= 10^{-3}$'
label_names_dict['UKP_dist_RBF_1.000000e+00_1.000000e-03'] = 'UKP_RBF, $\lambda = 1,\sigma= 10^{-3}$'
label_names_dict['UKP_dist_RBF_1.000000e+01_1.000000e-03'] = 'UKP_RBF, $\lambda = 10,\sigma= 10^{-3}$'
label_names_dict['UKP_dist_RBF_1.000000e+02_1.000000e-03'] = 'UKP_RBF, $\lambda = 100,\sigma= 10^{-3}$'

label_names_dict['UKP_dist_RBF_1.000000e-03_1.000000e-02'] = 'UKP_RBF, $\lambda = 10^{-3},\sigma= 10^{-2}$'
label_names_dict['UKP_dist_RBF_1.000000e-02_1.000000e-02'] = 'UKP_RBF, $\lambda = 10^{-2},\sigma= 10^{-2}$'
label_names_dict['UKP_dist_RBF_1.000000e-01_1.000000e-02'] = 'UKP_RBF, $\lambda = 10^{-1},\sigma= 10^{-2}$'
label_names_dict['UKP_dist_RBF_1.000000e+00_1.000000e-02'] = 'UKP_RBF, $\lambda = 1,\sigma= 10^{-2}$'
label_names_dict['UKP_dist_RBF_1.000000e+01_1.000000e-02'] = 'UKP_RBF, $\lambda = 10,\sigma= 10^{-2}$'
label_names_dict['UKP_dist_RBF_1.000000e+02_1.000000e-02'] = 'UKP_RBF, $\lambda = 100,\sigma= 10^{-2}$'

label_names_dict['UKP_dist_RBF_1.000000e-03_1.000000e-01'] = 'UKP_RBF, $\lambda = 10^{-3},\sigma= 10^{-1}$'
label_names_dict['UKP_dist_RBF_1.000000e-02_1.000000e-01'] = 'UKP_RBF, $\lambda = 10^{-2},\sigma= 10^{-1}$'
label_names_dict['UKP_dist_RBF_1.000000e-01_1.000000e-01'] = 'UKP_RBF, $\lambda = 10^{-1},\sigma= 10^{-1}$'
label_names_dict['UKP_dist_RBF_1.000000e+00_1.000000e-01'] = 'UKP_RBF, $\lambda = 1,\sigma= 10^{-1}$'
label_names_dict['UKP_dist_RBF_1.000000e+01_1.000000e-01'] = 'UKP_RBF, $\lambda = 10,\sigma= 10^{-1}$'
label_names_dict['UKP_dist_RBF_1.000000e+02_1.000000e-01'] = 'UKP_RBF, $\lambda = 100,\sigma= 10^{-1}$'

label_names_dict['UKP_dist_RBF_1.000000e-03_1.000000e+00'] = 'UKP_RBF, $\lambda = 10^{-3},\sigma= 1$'
label_names_dict['UKP_dist_RBF_1.000000e-02_1.000000e+00'] = 'UKP_RBF, $\lambda = 10^{-2},\sigma= 1$'
label_names_dict['UKP_dist_RBF_1.000000e-01_1.000000e+00'] = 'UKP_RBF, $\lambda = 10^{-1},\sigma= 1$'
label_names_dict['UKP_dist_RBF_1.000000e+00_1.000000e+00'] = 'UKP_RBF, $\lambda = 1,\sigma= 1$'
label_names_dict['UKP_dist_RBF_1.000000e+01_1.000000e+00'] = 'UKP_RBF, $\lambda = 10,\sigma= 1$'
label_names_dict['UKP_dist_RBF_1.000000e+02_1.000000e+00'] = 'UKP_RBF, $\lambda = 100,\sigma= 1$'

label_names_dict['UKP_dist_RBF_1.000000e-03_1.000000e+01'] = 'UKP_RBF, $\lambda = 10^{-3},\sigma= 10$'
label_names_dict['UKP_dist_RBF_1.000000e-02_1.000000e+01'] = 'UKP_RBF, $\lambda = 10^{-2},\sigma= 10$'
label_names_dict['UKP_dist_RBF_1.000000e-01_1.000000e+01'] = 'UKP_RBF, $\lambda = 10^{-1},\sigma= 10$'
label_names_dict['UKP_dist_RBF_1.000000e+00_1.000000e+01'] = 'UKP_RBF, $\lambda = 1,\sigma= 10$'
label_names_dict['UKP_dist_RBF_1.000000e+01_1.000000e+01'] = 'UKP_RBF, $\lambda = 10,\sigma= 10$'
label_names_dict['UKP_dist_RBF_1.000000e+02_1.000000e+01'] = 'UKP_RBF, $\lambda = 100,\sigma= 10$'

label_names_dict['UKP_dist_RBF_1.000000e-03_0.000000e+00'] = 'UKP_RBF, $\lambda = 10^{-3},\sigma= 100$'
label_names_dict['UKP_dist_RBF_1.000000e-02_0.000000e+00'] = 'UKP_RBF, $\lambda = 10^{-2},\sigma= 100$'
label_names_dict['UKP_dist_RBF_1.000000e-01_0.000000e+00'] = 'UKP_RBF, $\lambda = 10^{-1},\sigma= 100$'
label_names_dict['UKP_dist_RBF_1.000000e+00_0.000000e+00'] = 'UKP_RBF, $\lambda = 1,\sigma= 100$'
label_names_dict['UKP_dist_RBF_1.000000e+01_0.000000e+00'] = 'UKP_RBF, $\lambda = 10,\sigma= 100$'
label_names_dict['UKP_dist_RBF_1.000000e+02_0.000000e+00'] = 'UKP_RBF, $\lambda = 100,\sigma= 100$'

label_names_dict['UKP_dist_Laplace_0.001_1e-07'] = 'UKP_Lap, $\lambda = 10^{-3},\sigma= 10^{-7}$'
label_names_dict['UKP_dist_Laplace_0.01_1e-07'] = 'UKP_Lap, $\lambda = 10^{-2},\sigma= 10^{-7}$'
label_names_dict['UKP_dist_Laplace_0.09999999999999999_1e-07'] = 'UKP_Lap, $\lambda = 10^{-1},\sigma= 10^{-7}$'
label_names_dict['UKP_dist_Laplace_1.0_1e-07'] = 'UKP_Lap, $\lambda = 1,\sigma= 10^{-7}$'
label_names_dict['UKP_dist_Laplace_10.0_1e-07'] = 'UKP_Lap, $\lambda = 10,\sigma= 10^{-7}$'
label_names_dict['UKP_dist_Laplace_100.0_1e-07'] = 'UKP_Lap, $\lambda = 100,\sigma= 10^{-7}$'

label_names_dict['UKP_dist_Laplace_0.001_1e-06'] = 'UKP_Lap, $\lambda = 10^{-3},\sigma= 10^{-6}$'
label_names_dict['UKP_dist_Laplace_0.01_1e-06'] = 'UKP_Lap, $\lambda = 10^{-2},\sigma= 10^{-6}$'
label_names_dict['UKP_dist_Laplace_0.09999999999999999_1e-06'] = 'UKP_Lap, $\lambda = 10^{-1},\sigma= 10^{-6}$'
label_names_dict['UKP_dist_Laplace_1.0_1e-06'] = 'UKP_Lap, $\lambda = 1,\sigma= 10^{-6}$'
label_names_dict['UKP_dist_Laplace_10.0_1e-06'] = 'UKP_Lap, $\lambda = 10,\sigma= 10^{-6}$'
label_names_dict['UKP_dist_Laplace_100.0_1e-06'] = 'UKP_Lap, $\lambda = 100,\sigma= 10^{-6}$'

label_names_dict['UKP_dist_Laplace_0.001_1e-05'] = 'UKP_Lap, $\lambda = 10^{-3},\sigma= 10^{-5}$'
label_names_dict['UKP_dist_Laplace_0.01_1e-05'] = 'UKP_Lap, $\lambda = 10^{-2},\sigma= 10^{-5}$'
label_names_dict['UKP_dist_Laplace_0.09999999999999999_1e-05'] = 'UKP_Lap, $\lambda = 10^{-1},\sigma= 10^{-5}$'
label_names_dict['UKP_dist_Laplace_1.0_1e-05'] = 'UKP_Lap, $\lambda = 1,\sigma= 10^{-5}$'
label_names_dict['UKP_dist_Laplace_10.0_1e-05'] = 'UKP_Lap, $\lambda = 10,\sigma= 10^{-5}$'
label_names_dict['UKP_dist_Laplace_100.0_1e-05'] = 'UKP_Lap, $\lambda = 100,\sigma= 10^{-5}$'

label_names_dict['UKP_dist_Laplace_0.001_1e-04'] = 'UKP_Lap, $\lambda = 10^{-3},\sigma= 10^{-4}$'
label_names_dict['UKP_dist_Laplace_0.01_1e-04'] = 'UKP_Lap, $\lambda = 10^{-2},\sigma= 10^{-4}$'
label_names_dict['UKP_dist_Laplace_0.09999999999999999_1e-04'] = 'UKP_Lap, $\lambda = 10^{-1},\sigma= 10^{-4}$'
label_names_dict['UKP_dist_Laplace_1.0_1e-04'] = 'UKP_Lap, $\lambda = 1,\sigma= 10^{-4}$'
label_names_dict['UKP_dist_Laplace_10.0_1e-04'] = 'UKP_Lap, $\lambda = 10,\sigma= 10^{-4}$'
label_names_dict['UKP_dist_Laplace_100.0_1e-04'] = 'UKP_Lap, $\lambda = 100,\sigma= 10^{-4}$'

label_names_dict['UKP_dist_Laplace_0.001_1e-03'] = 'UKP_Lap, $\lambda = 10^{-3},\sigma= 10^{-3}$'
label_names_dict['UKP_dist_Laplace_0.01_1e-03'] = 'UKP_Lap, $\lambda = 10^{-2},\sigma= 10^{-3}$'
label_names_dict['UKP_dist_Laplace_0.09999999999999999_1e-03'] = 'UKP_Lap, $\lambda = 10^{-1},\sigma= 10^{-3}$'
label_names_dict['UKP_dist_Laplace_1.0_1e-03'] = 'UKP_Lap, $\lambda = 1,\sigma= 10^{-3}$'
label_names_dict['UKP_dist_Laplace_10.0_1e-03'] = 'UKP_Lap, $\lambda = 10,\sigma= 10^{-3}$'
label_names_dict['UKP_dist_Laplace_100.0_1e-03'] = 'UKP_Lap, $\lambda = 100,\sigma= 10^{-3}$'

label_names_dict['UKP_dist_Laplace_0.001_1e-02'] = 'UKP_Lap, $\lambda = 10^{-3},\sigma= 10^{-2}$'
label_names_dict['UKP_dist_Laplace_0.01_1e-02'] = 'UKP_Lap, $\lambda = 10^{-2},\sigma= 10^{-2}$'
label_names_dict['UKP_dist_Laplace_0.09999999999999999_1e-02'] = 'UKP_Lap, $\lambda = 10^{-1},\sigma= 10^{-2}$'
label_names_dict['UKP_dist_Laplace_1.0_1e-02'] = 'UKP_Lap, $\lambda = 1,\sigma= 10^{-2}$'
label_names_dict['UKP_dist_Laplace_10.0_1e-02'] = 'UKP_Lap, $\lambda = 10,\sigma= 10^{-2}$'
label_names_dict['UKP_dist_Laplace_100.0_1e-02'] = 'UKP_Lap, $\lambda = 100,\sigma= 10^{-2}$'

label_names_dict['UKP_dist_Laplace_0.001_1e-01'] = 'UKP_Lap, $\lambda = 10^{-3},\sigma= 10^{-1}$'
label_names_dict['UKP_dist_Laplace_0.01_1e-01'] = 'UKP_Lap, $\lambda = 10^{-2},\sigma= 10^{-1}$'
label_names_dict['UKP_dist_Laplace_0.09999999999999999_1e-01'] = 'UKP_Lap, $\lambda = 10^{-1},\sigma= 10^{-1}$'
label_names_dict['UKP_dist_Laplace_1.0_1e-01'] = 'UKP_Lap, $\lambda = 1,\sigma= 10^{-1}$'
label_names_dict['UKP_dist_Laplace_10.0_1e-01'] = 'UKP_Lap, $\lambda = 10,\sigma= 10^{-1}$'
label_names_dict['UKP_dist_Laplace_100.0_1e-01'] = 'UKP_Lap, $\lambda = 100,\sigma= 10^{-1}$'

label_names_dict['UKP_dist_Laplace_0.001_1.0'] = 'UKP_Lap, $\lambda = 10^{-3},\sigma= 1$'
label_names_dict['UKP_dist_Laplace_0.01_1.0'] = 'UKP_Lap, $\lambda = 10^{-2},\sigma= 1$'
label_names_dict['UKP_dist_Laplace_0.09999999999999999_1.0'] = 'UKP_Lap, $\lambda = 10^{-1},\sigma= 1$'
label_names_dict['UKP_dist_Laplace_1.0_1.0'] = 'UKP_Lap, $\lambda = 1,\sigma= 1$'
label_names_dict['UKP_dist_Laplace_10.0_1.0'] = 'UKP_Lap, $\lambda = 10,\sigma= 1$'
label_names_dict['UKP_dist_Laplace_100.0_1.0'] = 'UKP_Lap, $\lambda = 100,\sigma= 1$'

label_names_dict['UKP_dist_Laplace_0.001_10.0'] = 'UKP_Lap, $\lambda = 10^{-3},\sigma= 10$'
label_names_dict['UKP_dist_Laplace_0.01_10.0'] = 'UKP_Lap, $\lambda = 10^{-2},\sigma= 10$'
label_names_dict['UKP_dist_Laplace_0.09999999999999999_10.0'] = 'UKP_Lap, $\lambda = 10^{-1},\sigma= 10$'
label_names_dict['UKP_dist_Laplace_1.0_10.0'] = 'UKP_Lap, $\lambda = 1,\sigma= 10$'
label_names_dict['UKP_dist_Laplace_10.0_10.0'] = 'UKP_Lap, $\lambda = 10,\sigma= 10$'
label_names_dict['UKP_dist_Laplace_100.0_10.0'] = 'UKP_Lap, $\lambda = 100,\sigma= 10$'

label_names_dict['UKP_dist_Laplace_0.001_0.0'] = 'UKP_Lap, $\lambda = 10^{-3},\sigma= 100$'
label_names_dict['UKP_dist_Laplace_0.01_0.0'] = 'UKP_Lap, $\lambda = 10^{-2},\sigma= 100$'
label_names_dict['UKP_dist_Laplace_0.09999999999999999_0.0'] = 'UKP_Lap, $\lambda = 10^{-1},\sigma= 100$'
label_names_dict['UKP_dist_Laplace_1.0_0.0'] = 'UKP_Lap, $\lambda = 1,\sigma= 100$'
label_names_dict['UKP_dist_Laplace_10.0_0.0'] = 'UKP_Lap, $\lambda = 10,\sigma= 100$'
label_names_dict['UKP_dist_Laplace_100.0_0.0'] = 'UKP_Lap, $\lambda = 100,\sigma= 100$'

# Total 35 models

classes = {}
for c in all_classes:
    classes[c] = all_classes[c]
num_classes = len(classes)
class_names = np.array(list(classes.keys()))

models_pretrained_folder = "/content/drive/MyDrive/UKP/imagenet_experiments/models/"

pretrained_model_names_available = np.sort([x[:-4] for x in os.listdir(f"{models_pretrained_folder}")])
pretrained_model_names = []

for name in pretrained_model_names_available :
    if "rm" not in name:
        pretrained_model_names.append(name)

pretrained_model_names_categorized = ['regnet_x_16gf', 'regnet_x_1_6gf', 'regnet_x_32gf', 'regnet_x_3_2gf', 'regnet_x_400mf',
            'regnet_x_800mf', 'regnet_x_8gf', 'regnet_y_16gf', 'regnet_y_1_6gf', 'regnet_y_32gf',
            'regnet_y_3_2gf', 'regnet_y_400mf', 'regnet_y_800mf', 'regnet_y_8gf', 'resnet18',
            'resnext50_32x4d', 'wide_resnet50_2','efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4',
            'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7','alexnet','googlenet','inception','mnasnet','vgg16','mobilenet_v2', 'mobilenet_v3_large', 'mobilenet_v3_small',
               'convnext_small', 'convnext_tiny']

pretrained_model_names = np.array(pretrained_model_names_categorized)
num_pretrained_models = len(pretrained_model_names)

short_pretrained_model_names = pretrained_model_names_categorized

# loading test distance file and seeing its entries
datafilepath = "/content/drive/MyDrive/UKP/imagenet_experiments/distances_torch/val/pretrained/alexnet_pretrained_rep_convnext_small_pretrained_rep.npz"
data = np.load(f"{datafilepath}")
print(data.files)

distnames = data.files

####################################################################################################################3

# 0.1. Show distance matrices for different distance functions
subset = "val"
distances_pretrained_folder = "/home/soumya/PycharmProjects/UKP/imagenet_experiments/distances_torch/val/pretrained"
distances_pretrained_categorized_folder = "/home/soumya/PycharmProjects/UKP/imagenet_experiments/distances_torch/val/pretrained_categorized"

figures_folder = "/home/soumya/PycharmProjects/UKP/imagenet_experiments/distances_torch/val/Figures"

dist_array = np.load(f'{distances_pretrained_categorized_folder}/all_distances_categorized_correctedfinal.npz')

sorted_inds = np.array([], dtype=int)
spacings = []
for name in class_names:
    class_inds = np.where(np.in1d(short_pretrained_model_names, classes[name]))[0]
    spacings.append(len(sorted_inds) + len(class_inds) / 2)
    sorted_inds = np.append(sorted_inds, class_inds)
spacings = np.array(spacings)

for distname in distnames:
    try:
      D_temp = dist_array[distname]
      D = np.zeros((num_pretrained_models, num_pretrained_models))
      row_indices, col_indices = np.triu_indices(num_pretrained_models, k=1)
      D[row_indices, col_indices] = D_temp
      D = symmetrize(D)
      if "GULP" in distname or "UKP" in distname:
          D[D < 0] = np.nan

      fig, ax = plt.subplots(figsize=(8, 8))
      im = ax.pcolormesh(D)
      divider = make_axes_locatable(ax)
      cax = divider.append_axes("right", size='5%', pad=0.05)
      fig.colorbar(im, cax=cax, orientation='vertical')

      ax.set_title(f"Distance matrix for {label_names_dict[distname]}", fontsize=20)
      ax.set_yticks(spacings + 0.5)
      ax.set_yticklabels(class_names, fontsize=15)
      ax.set_ylabel("Networks")
      ax.set_xticks(spacings + 0.5)
      ax.set_xticklabels(class_names, rotation=90, fontsize=13)
      ax.set_xlabel("Networks")

      plt.tight_layout()
      plt.savefig(f"{figures_folder}/Heatmaps/Heatmap for {distname}.png", format='png', dpi=300,
                  bbox_inches='tight')
      plt.savefig(f"{figures_folder}/Heatmaps/Heatmap for {distname}.pdf", format='pdf', dpi=300,
                  bbox_inches='tight')
      plt.close()
    except:
      print("Skipped due to Error")

# 0.2 Dendogram and TSNE

pretrained_subset_inds = []
for j in range(num_classes):
    class_inds = np.where(np.in1d(short_pretrained_model_names, classes[class_names[j]]))[0]
    pretrained_subset_inds.append(class_inds)
pretrained_subset_inds = np.sort(np.hstack(pretrained_subset_inds))
num_pretrained_subset = len(pretrained_subset_inds)

subset = "val"
num_dists = len(distnames)

dist_array = np.load("/home/soumya/PycharmProjects/UKP/imagenet_experiments/distances_torch/val/pretrained_categorized/all_distances_categorized_correctedfinal.npz")

embed_type = "TSNE"

pretrained_embeddings = np.zeros((num_dists, num_pretrained_subset, 2))
for i in range(num_dists):
    print(f"Pretrained Networks: {distnames[i]}")
    D_temp = dist_array[distnames[i]]
    D = np.zeros((num_pretrained_models, num_pretrained_models))
    row_indices, col_indices = np.triu_indices(num_pretrained_models, k=1)
    D[row_indices, col_indices] = D_temp
    D = symmetrize(D)
    D_subset = D[pretrained_subset_inds, :][:, pretrained_subset_inds]

    try:
        if embed_type == "TSNE":
            X_embedded = TSNE(n_components=2, perplexity=20.0, init="random", metric="precomputed").fit_transform(
                D_subset)
        elif embed_type == "UMAP":
            X_embedded = umap.UMAP(n_components=2, n_neighbors=20, min_dist=0.1).fit_transform(np.sqrt(D_subset))
        elif embed_type == "MDS":
            X_embedded = MDS(n_components=2, dissimilarity="precomputed").fit_transform(np.sqrt(D_subset))
    except:
        X_embedded = np.full(pretrained_embeddings[i, :, :].shape,np.nan)
        print("Nan generated")

    pretrained_embeddings[i, :, :] = X_embedded


hierarchy.set_link_color_palette(['orange', 'brown', 'indigo', 'cyan'])

dist_inds = [i for i in range(len(distnames))]

labels = [distnames[i] for i in range(len(distnames))]

cmap = plt.cm.tab10
colors = [cmap(i) for i in range(num_classes)]


for i in range(len(distnames)):
    try:
        fig, axs = plt.subplots(2, 1, figsize=(30, 20))

        ax = axs[0]
        X_embedded = pretrained_embeddings[dist_inds[i], :, :]
        for j in range(num_classes):
            class_inds = np.array([i for i, s in enumerate(short_pretrained_model_names) if s in classes[class_names[j]]])
            ax.scatter(X_embedded[class_inds, 0], X_embedded[class_inds, 1], s=200, color=colors[j])
        ax.set_title(f"{embed_type} embedding", fontdict={'fontsize': 22})
        ax.legend(class_names, fontsize=15)

        ax = axs[1]
        D_temp = dist_array[distnames[i]]
        D = np.zeros((num_pretrained_models, num_pretrained_models))
        row_indices, col_indices = np.triu_indices(num_pretrained_models, k=1)
        D[row_indices, col_indices] = D_temp
        D = symmetrize(D)
        D_subset = D[pretrained_subset_inds, :][:, pretrained_subset_inds]

        Z = linkage(pdist(D_subset), 'ward')
        count_sort = 'ascending'

        dendrogram(Z, labels=short_pretrained_model_names, leaf_rotation=90, leaf_font_size=20,
                   count_sort=count_sort, ax=ax)
        ax.set_title("Dendrogram", fontsize=25)

        plt.subplots_adjust(hspace=0.3)

        plt.savefig(f"{figures_folder}/DendogramandTSNE/DendogramandTSNE for {distnames[i]}.pdf", format='pdf', dpi=300,
                    bbox_inches='tight')
        plt.savefig(f"{figures_folder}/DendogramandTSNE/DendogramandTSNE for {distnames[i]}.png", format='png', dpi=300,
                    bbox_inches='tight')
        plt.close()
    except:
        print("Skipped due to error")

# Only dendrogram

for i in range(len(distnames)):
    try:
        fig, ax = plt.subplots(1, 1, figsize=(25, 15))

        D_temp = dist_array[distnames[i]]
        D = np.zeros((num_pretrained_models, num_pretrained_models))
        row_indices, col_indices = np.triu_indices(num_pretrained_models, k=1)
        D[row_indices, col_indices] = D_temp
        D = symmetrize(D)
        D_subset = D[pretrained_subset_inds, :][:, pretrained_subset_inds]

        Z = linkage(pdist(D_subset), 'ward')
        count_sort = 'ascending'

        dendrogram(Z, labels=short_pretrained_model_names, leaf_rotation=90, leaf_font_size=20,
                    count_sort=count_sort, ax=ax)
        ax.set_title(f"Dendrogram for {label_names_dict[distnames[i]]}", fontsize = 40)

        plt.subplots_adjust(hspace=0.3)

        plt.savefig(f"{figures_folder}/Dendogram/Dendogram for {distnames[i]}.pdf", format='pdf',
                    dpi=300,
                    bbox_inches='tight')
        plt.savefig(f"{figures_folder}/Dendogram/Dendogram for {distnames[i]}.png", format='png',
                    dpi=300,
                    bbox_inches='tight')
        plt.close()
    except:
        print("Skipped due to Error")



# Correlation plots

GULP_dists = [s for s in distnames if "GULP" in s]
UKP_dists = [s for s in distnames if "UKP" in s and "Laplace" not in s]
CKA_dists = [s for s in distnames if "CKA" in s and "Laplace" not in s]

subset_inds = range(35)

def vectorize_upper_triangle(A, subset_inds):
    assert (A.shape[0] == A.shape[1])
    if subset_inds is None:
        subset_inds = range(len(A.shape[0]))
    n = len(subset_inds)
    v = []
    for i in range(n):
        for j in range(i + 1, n):
            c_val = A[subset_inds[i], subset_inds[j]]
            v.append(c_val)
    return np.asarray(v)


def compare_d1_to_d2(d1, d2, subset_inds=None, d1_name=None, d2_name=None, fsz=17):
    D_temp = dist_array[d1]
    num_pretrained_models = 35
    D = np.zeros((num_pretrained_models, num_pretrained_models))
    row_indices, col_indices = np.triu_indices(num_pretrained_models, k=1)
    D[row_indices, col_indices] = D_temp
    D = symmetrize(D)

    d1_mat = D

    if subset_inds is None:
        subset_inds = range(d1_mat.shape[0])

    d1_vec = vectorize_upper_triangle(d1_mat, subset_inds)

    plt.subplots(1, 1, figsize=(4, 3))

    D_temp = dist_array[d2]
    D = np.zeros((num_pretrained_models, num_pretrained_models))
    row_indices, col_indices = np.triu_indices(num_pretrained_models, k=1)
    D[row_indices, col_indices] = D_temp
    D = symmetrize(D)

    d2_mat = D
    d2_vec = vectorize_upper_triangle(d2_mat, subset_inds)

    good_inds = np.asarray([i for i in range(len(d2_vec)) if d1_vec[i] > 0 and d2_vec[i] > 0])

    if len(good_inds) < len(d2_vec):
        print('WARNING: some distances that are < 0 were removed')

    d1_filtered = d1_vec[good_inds]
    d2_filtered = d2_vec[good_inds]

    plt.scatter(d2_filtered, d1_filtered, marker='.', s=5)
    #         plt.axis('equal')
    if d2_name is None:
        plt.xlabel(d2, fontsize=fsz)
    else:
        plt.xlabel(d2_name, fontsize=fsz)
    if d1_name is None:
        plt.ylabel(d1, fontsize=fsz)
    else:
        plt.ylabel(d1_name, fontsize=fsz)

    plt.yticks(fontsize=fsz - 2)
    plt.xticks(fontsize=fsz - 2)

    pearsonr = scipy.stats.pearsonr(d2_filtered, d1_filtered)
    plt.title(f"Correlation: {pearsonr[0]:.3f}", fontsize=fsz - 2)

    plt.tight_layout(pad=3.0)
    plt.savefig(f"{figures_folder}/Correlation/Correlation plot for {d1} and {d2}.pdf", format='pdf', dpi=300,
                bbox_inches='tight')
    plt.savefig(f"{figures_folder}/Correlation/Correlation plot for {d1} and {d2}.png", format='png', dpi=300,
                bbox_inches='tight')
    plt.close()


def compare_d1_to_d2_cleaned(d1, d2, subset_inds=None, d1_name=None, d2_name=None, fsz=15):
    D_temp = dist_array[d1]
    num_pretrained_models = 35
    D = np.zeros((num_pretrained_models, num_pretrained_models))
    row_indices, col_indices = np.triu_indices(num_pretrained_models, k=1)
    D[row_indices, col_indices] = D_temp
    D = symmetrize(D)

    d1_mat = D

    if subset_inds is None:
        subset_inds = range(d1_mat.shape[0])

    d1_vec = vectorize_upper_triangle(d1_mat, subset_inds)

    D_temp = dist_array[d2]
    D = np.zeros((num_pretrained_models, num_pretrained_models))
    row_indices, col_indices = np.triu_indices(num_pretrained_models, k=1)
    D[row_indices, col_indices] = D_temp
    D = symmetrize(D)

    d2_mat = D
    d2_vec = vectorize_upper_triangle(d2_mat, subset_inds)

    good_inds = np.asarray([i for i in range(len(d2_vec)) if d1_vec[i] > 0 and d2_vec[i] > 0])

    if len(good_inds) < len(d2_vec):
        print('WARNING: some distances that are < 0 were removed')

    d1_filtered = d1_vec[good_inds]
    d2_filtered = d2_vec[good_inds]

    plt.figure(figsize=(8,8))
    plt.scatter(d2_filtered, d1_filtered, marker='.', s=7)

    plt.xlabel(label_names_dict[d2], fontsize=fsz)
    plt.ylabel(label_names_dict[d1], fontsize=fsz)

    plt.yticks(fontsize=fsz - 2)
    plt.xticks(fontsize=fsz - 2)

    pearsonr = scipy.stats.pearsonr(d2_filtered, d1_filtered)
    plt.title(f"Correlation: {pearsonr[0]:.3f}", fontsize=fsz - 2)

    plt.tight_layout(pad=3.0)
    plt.savefig(f"{figures_folder}/Correlation/Correlation plot for {d1} and {d2}.pdf", format='pdf', dpi=300,
                bbox_inches='tight')
    plt.savefig(f"{figures_folder}/Correlation/Correlation plot for {d1} and {d2}.png", format='png', dpi=300,
                bbox_inches='tight')
    plt.close()



for i in range(0, len(distnames)):
    for j in range(i + 1, len(distnames)):
        try:
            d1 = distnames[i]
            d2 = distnames[j]
            if ((d1 in GULP_dists) or (d1 in UKP_dists) or (d1 in CKA_dists)) and ((d2 in GULP_dists) or (d2 in UKP_dists) or (d2 in CKA_dists)):
                print(f'({i} {d1},{j} {d2})')
                compare_d1_to_d2(d1, d2, None)
        except:
            print("Skipped due to Error")





